# Data Frame 이용해서 학습시키기

In [1]:
import tensorflow as tf 
from tensorflow.keras import layers

from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (28, 28, 1)
num_classes = 10

## Preprocess

In [3]:
import pandas as pd

In [4]:
train_df = pd.read_csv('./csv/train_dataset.csv')
train_df.head()

,name,class
0,dataset/cifar/train\0_frog.png,frog
1,dataset/cifar/train\10000_automobile.png,automobile
2,dataset/cifar/train\10001_frog.png,frog
3,dataset/cifar/train\10002_frog.png,frog
4,dataset/cifar/train\10003_ship.png,ship


In [5]:
test_df = pd.read_csv('./csv/test_dataset.csv')
test_df.head()

,name,class
0,dataset/cifar/test\0_cat.png,cat
1,dataset/cifar/test\1000_dog.png,dog
2,dataset/cifar/test\1001_airplane.png,airplane
3,dataset/cifar/test\1002_ship.png,ship
4,dataset/cifar/test\1003_deer.png,deer


In [7]:
train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale = 1./255.
)

In [8]:
train_generator = train_datagen.flow_from_dataframe(    # from directory 말고 from dataframe
    train_df,
    x_col = 'name', # 경로 설정 (filename이 기본값. 그래서 만들때 filename으로 써놓는게 베스트)
    y_col = 'class', # class축 설정 (class가 기본값)
    target_size = input_shape[:2],# 채널은 필요없다는
    batch_size = batch_size
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col = 'name',
    y_col = 'class',
    target_size = input_shape[:2],# 채널은 필요없다는
    batch_size = batch_size
)

Found 50000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


## Build Model

In [9]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [10]:
model.compile(optimizer = 
    tf.keras.optimizers.Adam(learning_rate),
    loss = 'categorical_crossentropy',
    metrics=['accuracy'])

## Training

In [11]:
len(train_generator) ## len(train_df) // 32

1563

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs = num_epochs,
    validation_data = test_generator,
    validation_steps=len(test_generator)
)